In [1]:
import torch
import random
import time
import numpy as np
import sentencepiece as spm
from transformers import ReformerConfig, ReformerModelWithLMHead, ReformerTokenizer, EncoderDecoderConfig, EncoderDecoderModel
from torch.utils.data import DataLoader, Dataset

NUM_BATCHES = None
BATCH_SIZE = 20
LEARNING_RATE = 0.001 #1e-4 #1e-4
VALIDATE_EVERY  = 10
SEQ_LEN = 4608

In [2]:
# spm.SentencePieceTrainer.Train("--input=./data/tokenizer_training/AAresiduals.txt \
#                                 --vocab_size=28 \
#                                 --model_prefix=sequence_tokenizer \
#                                 --model_type=char \
#                                 --character_coverage=1.0")
tokenizer = ReformerTokenizer(vocab_file="sequence_tokenizer.model", do_lower_case=False, model_max_length=SEQ_LEN)
tokenizer.max_model_input_sizes = SEQ_LEN

In [3]:

# def split_file(file,out1,out2,percentage=0.75,isShuffle=True,seed=42):
#     random.seed(seed)
#     with open(file, 'r',encoding="utf-8") as fin, open(out1, 'w') as foutBig, open(out2, 'w') as foutSmall:
#         nLines = sum(1 for line in fin)
#         fin.seek(0)

#         nTrain = int(nLines*percentage) 
#         nValid = nLines - nTrain

#         i = 0
#         for line in fin:
#             r = random.random() if isShuffle else 0 # so that always evaluated to true when not isShuffle
#             if (i < nTrain and r < percentage) or (nLines - i > nValid):
#                 foutBig.write(line)
#                 i += 1
#             else:
#                 foutSmall.write(line)
                
# split_file("data/yeast/yeast.txt", 
#            "data/yeast/yeast_train.txt",
#            "data/yeast/yeast_val.txt",
#            percentage=0.9)

In [4]:
class SequenceDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, tokenizer, _len):
        super().__init__()
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.tokenizer = tokenizer
        self._len = _len

    @classmethod
    def prepare_from_file(cls, file_path, tokenizer):
        
        with open(file_path) as file:
            
            X = [l.strip() for l in file]
            X = [tokenizer.encode(sequence)[1:tokenizer.max_len+1] for sequence in X]
            
            temp = [tokenizer.prepare_for_model(sequence) for sequence in X]
            
            input_ids = [np.pad(x["input_ids"], 
                                (0, tokenizer.max_len - len(x["input_ids"])), 
                                'constant', constant_values=0) for x in temp]

            attention_mask = [np.pad(x["attention_mask"], 
                                     (0, tokenizer.max_len - len(x["attention_mask"])),
                                     'constant', constant_values=0) for x in temp]
            
            labels = [np.pad(x["input_ids"], 
                             (0, tokenizer.max_len - len(x["input_ids"])), 
                             'constant', constant_values=-100) for x in temp]

            input_ids = [torch.tensor(x, dtype=torch.int64) for x in input_ids]
            attention_mask = [torch.tensor(x, dtype=torch.int64) for x in attention_mask]
            labels = [torch.tensor(x, dtype=torch.int64) for x in labels]
            
            input_ids = torch.stack([input_ids[i] for i in range(len(input_ids))]).squeeze()
            attention_mask = torch.stack([attention_mask[i] for i in range(len(attention_mask))]).squeeze()
            labels = torch.stack([labels[i] for i in range(len(labels))]).squeeze()
            
            del(temp); del(X);
        return cls(input_ids, attention_mask, labels, tokenizer, len(input_ids))

    def __getitem__(self, index):
        return {"input_ids": self.input_ids[index, ].cuda(), 
                "attention_mask": self.attention_mask[index, ].cuda(),
                "labels": self.labels[index, ].cuda()}

    def __len__(self):
        return self._len

In [5]:
def cycle(loader):
    while True:
        for data in loader:
            yield data

train_dataset = SequenceDataset.prepare_from_file("data/yeast/yeast_train.txt", tokenizer)
val_dataset = SequenceDataset.prepare_from_file("data/yeast/yeast_val.txt", tokenizer)
train_loader = cycle(DataLoader(train_dataset, batch_size=BATCH_SIZE))
val_loader = cycle(DataLoader(val_dataset, batch_size=BATCH_SIZE))


Token indices sequence length is longer than the specified maximum sequence length for this model (4911 > 4608). Running this sequence through the model will result in indexing errors


In [6]:
# next(train_loader)['input_ids'].shape

In [7]:
# next(train_loader)

In [8]:
# configuration = ReformerConfig.from_pretrained("google/reformer-crime-and-punishment")
# configuration.axial_pos_shape = (64, 72)
# configuration.max_position_embeddings=SEQ_LEN
# configuration.vocab_size=tokenizer.vocab_size
# configuration.save_pretrained('model/config/')
configuration = ReformerConfig.from_pretrained('model/config/')
model = ReformerModelWithLMHead(configuration)

model.cuda()

ReformerModelWithLMHead(
  (reformer): ReformerModel(
    (embeddings): ReformerEmbeddings(
      (word_embeddings): Embedding(28, 256)
      (position_embeddings): AxialPositionEmbeddings(
        (weights): ParameterList(
            (0): Parameter containing: [torch.cuda.FloatTensor of size 64x1x64 (GPU 0)]
            (1): Parameter containing: [torch.cuda.FloatTensor of size 1x72x192 (GPU 0)]
        )
      )
    )
    (encoder): ReformerEncoder(
      (layers): ModuleList(
        (0): ReformerLayer(
          (attention): ReformerAttention(
            (layer_norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (self_attention): LocalSelfAttention(
              (query): Linear(in_features=256, out_features=128, bias=False)
              (key): Linear(in_features=256, out_features=128, bias=False)
              (value): Linear(in_features=256, out_features=128, bias=False)
            )
            (output): ReformerSelfOutput(
              (dense): Linear(

In [9]:
NUM_BATCHES = len(train_dataset)//BATCH_SIZE
NUM_BATCHES
from transformers import AdamW
optimizer = AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [33]:
from collections import OrderedDict 
import json
import pandas as pd

all_training_loss = OrderedDict()
all_val_loss = OrderedDict()
argmax_to_save=[]

for x in range(40):
    print(f"epoch {x}")
    start = time.time()

    training_loss = OrderedDict()
    val_loss = OrderedDict()
    
    
    for i in range(NUM_BATCHES):
        print("step {}".format(i))
        model.train()

        tmp = next(train_loader)
        input_ids = tmp['input_ids']
        attention_mask = tmp['attention_mask']
        labels = tmp['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, prediction_scores = outputs[:2]
        loss.backward()
        
        training_loss[f"Epoch {x} Step {i}"] = loss.item()
        all_training_loss[f"Epoch {x} Step {i}"] = loss.item()
        print(f'training loss: {loss.item()}')

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()
        optimizer.zero_grad()

        if i % VALIDATE_EVERY == 0:
            model.eval()
            with torch.no_grad():
                tmp = next(val_loader)
                input_ids = tmp['input_ids']
                attention_mask = tmp['attention_mask']
                labels = tmp['labels']
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss, prediction_scores = outputs[:2]

                val_loss[f"Epoch {x} Step {i}"] = loss.item()
                all_val_loss[f"Epoch {x} Step {i}"] = loss.item()
                print(f'validation loss: {loss.item()}')
        
        if i%10 == 0:
            print('xxxxxxxxxxxxxx')
            y = torch.argmax(prediction_scores[0,:,:], dim=1).cpu().numpy() #batch x seq_len x num_tokens -> batch x seq_len
            argmax_to_save.append(y.tolist())
            print(type(argmax_to_save[0]))
            with open('saved_0624/argmax_saved.txt', 'w') as f:
                for ele in argmax_to_save:
                    f.write(str(ele)+'\n')

            del(y)

    torch.save(model.state_dict(), f"saved_0624/model/saved_model_epoch_{x}.pth")
    
    with open(f'saved_0624/saved_losses/training_loss_epoch_{x}.json', 'w') as f:
        f.write(json.dumps(training_loss))
    
    with open(f'saved_0624/saved_losses/val_loss_epoch_{x}.json', 'w') as f:
        f.write(json.dumps(val_loss))
    end = time.time()
    print(f"----------{(end-start)//60} min per epoch----------")
    

        
        

with open("saved_0624/saved_losses/training_loss_all.json", 'w') as f:
    f.write(json.dumps(all_training_loss))
with open("saved_0624/saved_losses/val_loss_all.json", 'w') as f:
    f.write(json.dumps(all_val_loss))

epoch 0
step 0
training loss: 2.8765008449554443
validation loss: 2.8412394523620605
xxxxxxxxxxxxxx
<class 'list'>
step 1
training loss: 2.884265184402466
step 2
training loss: 2.874288320541382
step 3
training loss: 2.856649160385132
step 4
training loss: 2.862579822540283
step 5
training loss: 2.8804471492767334
step 6
training loss: 2.8804876804351807
step 7
training loss: 2.8839030265808105
step 8
training loss: 2.868535041809082
step 9
training loss: 2.885714054107666
step 10
training loss: 2.8828983306884766
validation loss: 2.8691999912261963
xxxxxxxxxxxxxx
<class 'list'>
step 11
training loss: 2.844682216644287
step 12
training loss: 2.8486948013305664
step 13
training loss: 2.8649795055389404
step 14
training loss: 2.8672597408294678
step 15
training loss: 2.875302314758301
step 16
training loss: 2.8728182315826416
step 17
training loss: 2.873661756515503
step 18
training loss: 2.877591371536255
step 19
training loss: 2.873741388320923
step 20
training loss: 2.886409044265747


training loss: 2.873277425765991
step 170
training loss: 2.8667941093444824
validation loss: 2.830172061920166
xxxxxxxxxxxxxx
<class 'list'>
step 171
training loss: 2.8753302097320557
step 172
training loss: 2.85284161567688
step 173
training loss: 2.8603901863098145
step 174
training loss: 2.8387258052825928
step 175
training loss: 2.853752374649048
step 176
training loss: 2.860642671585083
step 177
training loss: 2.848814010620117
step 178
training loss: 2.8915841579437256
step 179
training loss: 2.8639743328094482
step 180
training loss: 2.855835437774658
validation loss: 2.8348782062530518
xxxxxxxxxxxxxx
<class 'list'>
step 181
training loss: 2.8677563667297363
step 182
training loss: 2.8688716888427734
step 183
training loss: 2.8593363761901855
step 184
training loss: 2.8514082431793213
step 185
training loss: 2.8339900970458984
step 186
training loss: 2.8253090381622314
step 187
training loss: 2.8785059452056885
step 188
training loss: 2.8570072650909424
step 189
training loss: 2

training loss: 2.8506546020507812
step 64
training loss: 2.8859617710113525
step 65
training loss: 2.8663883209228516
step 66
training loss: 2.8444063663482666
step 67
training loss: 2.8789453506469727
step 68
training loss: 2.8585915565490723
step 69
training loss: 2.8668394088745117
step 70
training loss: 2.8662843704223633
validation loss: 2.8824474811553955
xxxxxxxxxxxxxx
<class 'list'>
step 71
training loss: 2.8400392532348633
step 72
training loss: 2.877986431121826
step 73
training loss: 2.862908124923706
step 74
training loss: 2.8804614543914795
step 75
training loss: 2.8763272762298584
step 76
training loss: 2.897451400756836
step 77
training loss: 2.8809328079223633
step 78
training loss: 2.842601776123047
step 79
training loss: 2.879701852798462
step 80
training loss: 2.862748622894287
validation loss: 2.9303300380706787
xxxxxxxxxxxxxx
<class 'list'>
step 81
training loss: 2.855797290802002
step 82
training loss: 2.860722541809082
step 83
training loss: 2.859924077987671
ste

training loss: 2.8766047954559326
step 232
training loss: 2.8528780937194824
step 233
training loss: 2.8744068145751953
step 234
training loss: 2.8801419734954834
step 235
training loss: 2.8501930236816406
step 236
training loss: 2.8520689010620117
step 237
training loss: 2.8765742778778076
step 238
training loss: 2.865762948989868
step 239
training loss: 2.868450164794922
step 240
training loss: 2.8700549602508545
validation loss: 2.8671011924743652
xxxxxxxxxxxxxx
<class 'list'>
step 241
training loss: 2.8714592456817627
step 242
training loss: 2.8643758296966553
step 243
training loss: 2.8654067516326904
step 244
training loss: 2.8640589714050293
step 245
training loss: 2.8636748790740967
step 246
training loss: 2.860548734664917
step 247
training loss: 2.8278822898864746
step 248
training loss: 2.8555455207824707
step 249
training loss: 2.87644100189209
step 250
training loss: 2.855895757675171
validation loss: 2.8675179481506348
xxxxxxxxxxxxxx
<class 'list'>
step 251
training loss:

training loss: 2.892695665359497
step 128
training loss: 2.812361240386963
step 129
training loss: 2.864100456237793
step 130
training loss: 2.856313705444336
validation loss: 2.852213144302368
xxxxxxxxxxxxxx
<class 'list'>
step 131
training loss: 2.880898952484131
step 132
training loss: 2.8603715896606445
step 133
training loss: 2.884674549102783
step 134
training loss: 2.856658697128296
step 135
training loss: 2.8428714275360107
step 136
training loss: 2.8543310165405273
step 137
training loss: 2.8710803985595703
step 138
training loss: 2.8517658710479736
step 139
training loss: 2.882544755935669
step 140
training loss: 2.868260145187378
validation loss: 2.868175506591797
xxxxxxxxxxxxxx
<class 'list'>
step 141
training loss: 2.8277041912078857
step 142
training loss: 2.8546416759490967
step 143
training loss: 2.8589553833007812
step 144
training loss: 2.85919451713562
step 145
training loss: 2.871857166290283
step 146
training loss: 2.8695826530456543
step 147
training loss: 2.87129

...

training loss: 2.855757713317871
step 264
training loss: 2.8464958667755127
step 265
training loss: 2.866717576980591
step 266
training loss: 2.873157501220703
step 267
training loss: 2.846836566925049
step 268
training loss: 2.8722457885742188
step 269
training loss: 2.875079393386841
step 270
training loss: 2.8444576263427734
validation loss: 2.8553378582000732
xxxxxxxxxxxxxx
<class 'list'>
step 271
training loss: 2.8469693660736084
----------3.0 min per epoch----------
epoch 37
step 0
training loss: 2.869276285171509
validation loss: 2.8647539615631104
xxxxxxxxxxxxxx
<class 'list'>
step 1
training loss: 2.8628830909729004
step 2
training loss: 2.8613266944885254
step 3
training loss: 2.863750457763672
step 4
training loss: 2.864367723464966
step 5
training loss: 2.856848955154419
step 6
training loss: 2.8624696731567383
step 7
training loss: 2.859600782394409
step 8
training loss: 2.8477542400360107
step 9
training loss: 2.848332166671753
step 10
training loss: 2.8114852905273438
va

training loss: 2.8507537841796875
validation loss: 2.869924306869507
xxxxxxxxxxxxxx
<class 'list'>
step 161
training loss: 2.865739107131958
step 162
training loss: 2.8867757320404053
step 163
training loss: 2.8077688217163086
step 164
training loss: 2.856053590774536
step 165
training loss: 2.8512885570526123
step 166
training loss: 2.8727147579193115
step 167
training loss: 2.8548271656036377
step 168
training loss: 2.8763725757598877
step 169
training loss: 2.851839542388916
step 170
training loss: 2.8426380157470703
validation loss: 2.8732407093048096
xxxxxxxxxxxxxx
<class 'list'>
step 171
training loss: 2.8521971702575684
step 172
training loss: 2.8645410537719727
step 173
training loss: 2.8505218029022217
step 174
training loss: 2.871896505355835
step 175
training loss: 2.8572144508361816
step 176
training loss: 2.8220713138580322
step 177
training loss: 2.8499069213867188
step 178
training loss: 2.850280284881592
step 179
training loss: 2.8487496376037598
step 180
training loss:

training loss: 2.8652939796447754
step 55
training loss: 2.8755850791931152
step 56
training loss: 2.8751845359802246
step 57
training loss: 2.8690288066864014
step 58
training loss: 2.8872363567352295
step 59
training loss: 2.8663036823272705
step 60
training loss: 2.872039794921875
validation loss: 2.8631396293640137
xxxxxxxxxxxxxx
<class 'list'>
step 61
training loss: 2.882110118865967
step 62
training loss: 2.8772778511047363
step 63
training loss: 2.870805025100708
step 64
training loss: 2.8745086193084717
step 65
training loss: 2.859215259552002
step 66
training loss: 2.8373684883117676
step 67
training loss: 2.8626065254211426
step 68
training loss: 2.8741302490234375
step 69
training loss: 2.8421671390533447
step 70
training loss: 2.860039472579956
validation loss: 2.8275561332702637
xxxxxxxxxxxxxx
<class 'list'>
step 71
training loss: 2.8633384704589844
step 72
training loss: 2.8483896255493164
step 73
training loss: 2.880664110183716
step 74
training loss: 2.8550825119018555


training loss: 2.8391237258911133
step 223
training loss: 2.82356595993042
step 224
training loss: 2.815164089202881
step 225
training loss: 2.876331329345703
step 226
training loss: 2.8553380966186523
step 227
training loss: 2.8785111904144287
step 228
training loss: 2.852041482925415
step 229
training loss: 2.859849214553833
step 230
training loss: 2.8782379627227783
validation loss: 2.8767387866973877
xxxxxxxxxxxxxx
<class 'list'>
step 231
training loss: 2.858262777328491
step 232
training loss: 2.843719005584717
step 233
training loss: 2.8590409755706787
step 234
training loss: 2.843041181564331
step 235
training loss: 2.853226900100708
step 236
training loss: 2.844680070877075
step 237
training loss: 2.8417537212371826
step 238
training loss: 2.833395004272461
step 239
training loss: 2.855253219604492
step 240
training loss: 2.8565287590026855
validation loss: 2.8742165565490723
xxxxxxxxxxxxxx
<class 'list'>
step 241
training loss: 2.8456361293792725
step 242
training loss: 2.8710

training loss: 2.8458597660064697
step 120
training loss: 2.8537704944610596
validation loss: 2.819408655166626
xxxxxxxxxxxxxx
<class 'list'>
step 121
training loss: 2.856252670288086
step 122
training loss: 2.8743338584899902
step 123
training loss: 2.8588085174560547
step 124
training loss: 2.8762454986572266
step 125
training loss: 2.8738770484924316
step 126
training loss: 2.8783910274505615
step 127
training loss: 2.851677179336548
step 128
training loss: 2.8704214096069336
step 129
training loss: 2.8610620498657227
step 130
training loss: 2.853886365890503
validation loss: 2.8564889430999756
xxxxxxxxxxxxxx
<class 'list'>
step 131
training loss: 2.864698648452759
step 132
training loss: 2.838554859161377
step 133
training loss: 2.866264581680298
step 134
training loss: 2.871492624282837
step 135
training loss: 2.810499668121338
step 136
training loss: 2.8513383865356445
step 137
training loss: 2.8495984077453613
step 138
training loss: 2.8641164302825928
step 139
training loss: 2.

In [18]:
torch.argmax(prediction_scores, dim=2)[:,0:15]

tensor([[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 13, 13, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 12, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 12, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 13, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 12, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 13, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
        [20, 20, 20, 20, 12, 20, 20, 20, 20, 13, 13, 20, 13, 20, 20],
        [20, 20, 20,